In [19]:
# test the convergence of the Euler solver

In [1]:
from ppmpy import Euler
from ppmpy.initial_conditions import acoustic_pulse

In [2]:
import numpy as np

In [12]:
simulations = []
for nx in [32, 64, 128, 256]:
    dt = 1.0 / nx * 0.192
    e = Euler(nx, 0.5, fixed_dt=dt, init_cond=acoustic_pulse,
              bc_left_type="periodic", bc_right_type="periodic")
    e.evolve(0.24, verbose=False)
    simulations.append(e)

In [13]:
# now loop over pairs and coarsen the finer one and compute the norm

In [24]:
from itertools import pairwise
ivar = 1

In [25]:
for coarse, fine in pairwise(simulations):
    _, cd = fine.grid.coarsen(fine.U[:, ivar])
    err = coarse.grid.norm(coarse.U[:, ivar] - cd)
    print(f"{fine.grid.nx:3d} -> {coarse.grid.nx:3d} : {err}")

 64 ->  32 : 0.0034705261499779964
128 ->  64 : 0.0017416772011993949
256 -> 128 : 0.0009095979375101773
